In [ ]:
import ehtim as eh
import os

In [ ]:
os.environ["OMP_NUM_THREADS"] = "1"
os.getenv("OMP_NUM_THREADS")

In [ ]:
# Load in example image and array
im = eh.image.load_txt('../models/avery_sgra_eofn.txt')
eht = eh.array.load_txt('../arrays/EHT2017.txt')

In [ ]:
# create example observation uvfits file
tint_sec = 5
tadv_sec = 600
tstart_hr = 0
tstop_hr = 24
bw_hz = 4e9
obs = im.observe(eht, tint_sec, tadv_sec, tstart_hr, tstop_hr, bw_hz,
                 sgrscat=False, ampcal=True, phasecal=False)
obs.save_uvfits('../../example_survey_obs.uvfits')

In [ ]:
# create a mock SEFD error budget for this array
sites = list(obs.tkey.keys())
SEFD_errs = {}
for site in sites:
    SEFD_errs[site] = 0.1

print(SEFD_errs)

In [ ]:
# create a dict of non-varying imaging parameters, such as filenames
# see a full list of fixed parameters in the docs
params_fixed = eh.survey.create_params_fixed(infile='../../example_survey_obs.uvfits',
                                             outfile_base='example_survey',
                                             outpath='../../example_survey_output/',
                                             nproc=-1, #se all available cores, can be changed
                                             overwrite=True,
                                             niter_static=2,
                                             SEFD_error_budget= SEFD_errs,
                                             ttype='nfft')

print(params_fixed)

In [ ]:
# create a DataFrame of parameters to be surveyed over
psets = eh.survey.create_survey_psets(zbl=[0.6,0.7], sys_noise=[0,0.01], prior_fwhm=[50])
#psets = eh.survey.create_survey_psets(zbl=[0.6], sys_noise=[0], prior_fwhm=[50])

display(psets)

In [ ]:
# to run only a single pset, create an instance of the ParameterSet class and use the function run()
# should take ~1 minute to run

#pset = dict(psets.iloc[0])
#ps = eh.survey.ParameterSet(pset, params_fixed)


In [ ]:
#ps.run()

In [ ]:
# display output image
# won't necessarily be good as the default selection of hyperparameters is not optimized for anything
#ps.im_out.display();

In [ ]:
# to run all parameter sets, use the run_survey function
# should take ~1-2 minutes to run
eh.survey.run_survey(psets, params_fixed)

In [ ]:
ps.ttype